### 12.3.3　数値結果

In [ ]:
#【ナイル川の流量データにローカルレベルモデルを適用（時変MCMC：馬蹄分布）】

# 前処理
set.seed(123)
library(rstan)

# Stanの事前設定：コードのHDD保存、並列演算、グラフの縦横比
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())
theme_set(theme_get() + theme(aspect.ratio = 3/4))

# モデル：生成・コンパイル
stan_mod_out <- stan_model(file = "model12-1.stan")

# 平滑化：実行（サンプリング）
fit_stan <- sampling(object = stan_mod_out,
                     data = list(t_max = t_max, y = y, 
                                 miss = as.integer(is.na(y)),
                                 m0 = modtv$m0, C0 = modtv$C0[1, 1]),
                     pars = c("lambda", "W_sqrt", "V_sqrt"),
                     seed = 123
            )

# 結果の確認
oldpar <- par(no.readonly = TRUE); options(max.print = 99999)
print(fit_stan, probs = c(0.025, 0.5, 0.975))
options(oldpar)
traceplot(fit_stan, pars = c("W_sqrt", "V_sqrt"), alpha = 0.5)

# カルマンフィルタのモデルをコピーして修正
modtv_MCMC <- modtv
modtv_MCMC$X[ , 1] <- (summary(fit_stan)$summary[   1:100, "mean"] *
                       summary(fit_stan)$summary["W_sqrt", "mean"])^2
modtv_MCMC$V[1, 1] <- (summary(fit_stan)$summary["V_sqrt", "mean"])^2
as.vector(modtv_MCMC$X); modtv_MCMC$V

# カルマン平滑化
dlmSmoothed_obj <- dlmSmooth(y = y, mod = modtv_MCMC)

# 平滑化分布の平均
stv_MCMC <- dropFirst(dlmSmoothed_obj$s)

# プロット
ts.plot(cbind(y, stv_MCMC, stv),
        lty=c("solid", "solid", "dashed"),
        col=c("lightgray", "blue", "red"))

# 凡例
legend(legend = c("観測値", "平均（時変MCMC：馬蹄分布)", "平均（時変カルマン平滑化）"),
       lty = c("solid", "solid", "dashed"),
       col = c("lightgray", "blue", "red"),
       x = "topright", cex = 0.7)